# Substitution solving

In [ ]:
import json
import utils
import numpy as np
import solve



def vectorAbs(v):
    return np.array([abs(x) for x in v])

def posToString(pos):
    return ",".join(str(x) for x in pos)
def bindingStr(a, b):
    return "{} {}".format(*sorted([posToString(a), posToString(b)]))

In [ ]:
solveSpecPath = '../shapes/8-mer_polyominoes/8-mer_49.json'
with open(solveSpecPath, 'r') as f:
    data = f.read()
solveSpec = json.loads(data)
utils.calcCoordmapFromTop(solveSpec['bindings'])

In [ ]:
l = [-1,-2,3,4,5,6]
list(x > 0 for x in l).index(True)

In [ ]:
def getFullyAdressableRule(solveSpecPath):
    with open(solveSpecPath, 'r') as f:
        data = f.read()
    solveSpec = json.loads(data)
    coordMaps = utils.calcCoordmapFromTop(solveSpec['bindings'])
    assert len(coordMaps) == 1, "Found {} shapes".format(len(coordMaps))
    coordMap = coordMaps[0]
    coords = coordMap.values()

    print(solveSpec['bindings'])
    connectors = list(bindingStr(coordMap[b[0]], coordMap[b[2]]) for b in solveSpec['bindings'])

    # Find out which dimension has the fewest connectors
    dimCount = [0,0,0]
    dims = [
        np.array([1,0,0]),
        np.array([0,1,0]),
        np.array([0,0,1])
    ]

    for p in coords:
        for d in utils.getRuleOrder():
            neigbourPos = p+d
            if bindingStr(p, neigbourPos) in connectors:
                for i in range(3):
                    if np.array_equal(vectorAbs(d), dims[i]):
                        dimCount[i] += 1
                        break

    minDim = dims[dimCount.index(min(dimCount))]
    
    # Initialise empty species
    rule = []
    cubePosMap = {}
    for iCube, p in enumerate(coords):
        cubeType = []
        for i, d in enumerate(utils.getFaceRotations()):
            alignDir = d
            if not np.array_equal(utils.getRuleOrder()[i], minDim):
                alignDir = minDim
            cubeType.append({'color':0, 'alignDir': alignDir})
        rule.append(cubeType)
        cubePosMap[posToString(p)] = iCube

    print(connectors)
    # Set colors and alignment direcitons
    colorCounter = 1
    for iCube, p in enumerate(coords):
        found = False
        for iFace, d in enumerate(utils.getRuleOrder()):
            neigbourPos = p+d
            if bindingStr(p, neigbourPos) in connectors:
                found = True
                invDir = -d
                iFaceNeigh = list(np.array_equal(invDir, v) for v in utils.getRuleOrder()).index(True)
                iCubeNeigh = cubePosMap[posToString(neigbourPos)]
                
                rule[iCube][iFace]['color'] = colorCounter
                rule[iCubeNeigh][iFaceNeigh]['color'] = -colorCounter
                rule[iCubeNeigh][iFaceNeigh]['alignDir'] = rule[iCube][iFace]['alignDir']
                
                colorCounter += 1
        if not found:
            print("{} not found in connections".format(posToString(p)))
                
    rule = [[{
        'color': face['color'],
        'orientation': round(utils.getSignedAngle(
            utils.getFaceRotations()[i],
            face['alignDir'],
            utils.getRuleOrder()[i]
        )*(2/np.pi)+4) % 4
    } for i, face in enumerate(s)] for s in rule]
        
    return rule

rule = getFullyAdressableRule('../shapes/8-mer_polyominoes/8-mer_49.json')
#rule = getFullyAdressableRule('../shapes/robot.json')
print(solve.ruleToDec(rule))

rule

In [ ]:
list(np.array_equal(np.array([0,1,0]), v) for v in utils.getRuleOrder()).index(True)

In [ ]:
[coord for coord in shapes[0].T]

In [ ]:
set("{}, {}".format(b[0], b[2]) for b in solveSpec['bindings'])